# **Exemplo: Método de Newton Multivariado**

Para averiguar a teoria sobre o Método de Newton, vou usar um exemplo dado por Mirshawka na página 238:
$$
\begin{cases}
  f(x,y) = 2x^3 - y^2 - 1 = 0
  \\
  g(x,y) = xy^3 - y - 4 = 0
\end{cases}
$$

Começaremos com o ponto $p^{(0)}=(x_0,y_0)=(1.2,1.7)$. Para resolver os sistemas linares utilizaremos o ```numpy.linalg.solve```.

In [18]:
from metodo_newton import Newton
from numpy.linalg import solve

In [19]:
f = lambda x,y: 2*x**3 - y**2 - 1
g = lambda x,y: x*y**3 - y - 4
F = [f,g]

p0 = [1.2, 1.7]

Usaremos também a forma analítica da Jacobiana:

In [20]:
Jac = [
  [lambda x,y: 6*x**2, lambda x,y: -2*y],
  [lambda x,y: y**3, lambda x,y: 3*x*y**2-1]
]

Inclusive, sabemos que uma solução precisa é (1.2342744841145, 1.6615264667959)$^t$, então a usaremos para medir o erro absoluto.

In [21]:
sol = [1.2342744841145, 1.6615264667959]

In [22]:
newton = Newton(F, metodo_sistema_linear=solve)

p_estimado, info = newton.aplicar(p0, Jac, qntd_exata_passos=1, solucao_exata=sol)

print(p_estimado)

Causa da parada: quantidade exata de passos atingida
[[1.23487626]
 [1.66097968]]


Dada a solução exata acima, obtemos um excelente resultado. Iteremos mais algumas vezes:

In [23]:
p_estimado, info = newton.aplicar(p0, Jac, qntd_exata_passos=6, solucao_exata=sol)

from auxiliares.exibir import exibir

real = info["erro"]
taxa = [real[i+1]/real[i]**2 for i in range(len(real)-1)]

tabela = exibir(
  ["n", "||y||_inf", "||p-p_k||_inf", "taxa"], 
  [[i+1 for i in range(6)], info["erro"], info["erro real"], ['-']+real]
)

Causa da parada: quantidade exata de passos atingida
   n |   ||y||_inf |   ||p-p_k||_inf | taxa
-----+-------------+-----------------+------------------------
   1 | 0.0390203   |     0.000601779 | -
   2 | 0.000601588 |     1.91209e-07 | 0.03902031917591337
   3 | 1.91209e-07 |     1.59872e-14 | 0.0006015879635945571
   4 | 1.90212e-14 |     3.37508e-14 | 1.912091666665793e-07
   5 | 1.6312e-16  |     3.39728e-14 | 1.9021216107427664e-14
   6 | 8.2289e-17  |     3.39728e-14 | 1.631204854837885e-16


Além disso, podemos usar também um Jacobiano numérico no lugar de um exato. Nesse caso, iremos usar as diferenças progressivas, com $h=1e-4$ arbitrariamente.

In [24]:
from auxiliares.diferencasFinitas import DiferencasFinitas

diferencas_progressivas = DiferencasFinitas(h=1e-4).progressivas
newton = Newton(F, metodo_sistema_linear=solve, metodo_diferenciacao=diferencas_progressivas, h=1e-4)

p_estimado, info = newton.aplicar(p0, qntd_exata_passos=6, solucao_exata=sol)

real = info["erro"]
taxa = [real[i+1]/real[i]**2 for i in range(len(real)-1)]

tabela = exibir(
  ["n", "||y||_inf", "||p-p_k||_inf", "taxa"], 
  [[i+1 for i in range(6)], info["erro"], info["erro real"], ['-']+real]
)

Causa da parada: quantidade exata de passos atingida
   n |   ||y||_inf |   ||p-p_k||_inf | taxa
-----+-------------+-----------------+------------------------
   1 | 0.0390168   |     0.000599823 | -
   2 | 0.000599598 |     2.43712e-07 | 0.039016757863373726
   3 | 2.4369e-07  |     2.256e-11   | 0.0005995976985941751
   4 | 2.2592e-11  |     3.19744e-14 | 2.4368965188745964e-07
   5 | 1.93892e-15 |     3.39728e-14 | 2.2592014304317388e-11
   6 | 8.22838e-17 |     3.39728e-14 | 1.93891933294119e-15
